##  SMOS GH_sm L4 SM estimates (1km)
AMALGAMATED_PRM_v2
This Polynomia Regression Model Function is is to determine the relationship bwtween SMOS L3 SM estimates and MODIS LST(daily) and NDVI (16 Day) estimates over Ghana. This model will then be use to downscale the SMOS L3 SM estimates to High resolution(1Km) SM estimates over Ghana.

It was developed from the following models
http://localhost:8888/notebooks/Documents/PHD/RWESCK/PHD%20RESEARCH/Datasets/Downscaling/DS_Model/PRM%20MODEL_SMOS%20Downscaling.ipynb
http://localhost:8888/notebooks/Documents/PHD/RWESCK/PHD%20RESEARCH/Datasets/Downscaling/DS_Model/LRM%20MODEL_SMOS%20Downscaling.ipynb

In [ ]:
### v2 (working) UPDATE OF V1

## Import necessary modules
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import cartopy.crs as ccrs
import joblib
import hvplot.xarray  # Ensure hvplot is imported and connected to xarray
import hvplot.pandas


def develop_regression_model(smos_asc_file, smos_desc_file, ndvi_file, lst_file):
    ##########################################################################
    # Open datasets
    smos_asc = smos_asc_file
    smos_desc = smos_desc_file
    ndvi_asc = ndvi_asc_file
    ndvi_desc = ndvi_desc_file
    lst_asc = lst_asc_file
    lst_desc = lst_desc_file
    ghana_shp = gpd.read_file(ghana_shp_path)
    north_shp = gpd.read_file(north_sect_path)
    
    # Realign datasets
    ndvi_aligned_asc = ndvi_asc.reindex(time=lst_asc['time'])
    ndvi_aligned_desc = ndvi_desc.reindex(time=lst_desc['time'])
    smos_aligned_asc = smos_asc.reindex(time=lst_desc['time'])
    smos_aligned_desc = smos_desc.reindex(time=lst_desc['time'])
    
    ndvi_aligned_asc = ndvi_aligned_asc.ffill(dim='time')
    ndvi_aligned_desc = ndvi_aligned_desc.ffill(dim='time')

    ###########################################################################
    # Select variables to use
    asc_sm = smos_asc['Soil_Moisture_asc']
    desc_sm = smos_desc['Soil_Moisture_desc']
    asc_ndvi = ndvi_aligned_asc['asc_25km_16d_NDVI']
    desc_ndvi = ndvi_aligned_desc['desc_25km_16d_NDVI']
    asc_lst = lst_asc['asc_LST_Day_25km_Celsius']
    desc_lst = lst_desc['desc_LST_Night_25km_Celsius']

    # Convert to DataFrame
    sm_asc_df = asc_sm.to_dataframe().reset_index()
    sm_desc_df = desc_sm.to_dataframe().reset_index()
    ndvi_asc_df = asc_ndvi.to_dataframe().reset_index()
    ndvi_desc_df = desc_ndvi.to_dataframe().reset_index()
    lst_asc_df = asc_lst.to_dataframe().reset_index()
    lst_desc_df = desc_lst.to_dataframe().reset_index()

    ###########################################################################
    # Merge DataFrames on 'time', 'lat', and 'lon'
    merged_ndvi = pd.merge(ndvi_asc_df, ndvi_desc_df, on=['time', 'lat', 'lon'])
    merged_lst = pd.merge(lst_asc_df, lst_desc_df, on=['time', 'lat', 'lon'])
    merged_smos = pd.merge(sm_asc_df, sm_desc_df, on=['time', 'lat', 'lon'])
    merged_df1 = pd.merge(lst_asc_df, ndvi_asc_df, on=['time', 'lat', 'lon'])
    merged_df2 = pd.merge(sm_asc_df, sm_desc_df, on=['time', 'lat', 'lon'])

    ###########################################################################
    # Drop rows with NaN values in a specific column
    merged_df1 = merged_df1.dropna(subset=['asc_LST_Day_25km_Celsius', 'asc_25km_16d_NDVI'])
    merged_df2 = merged_df2.dropna(subset=['Soil_Moisture_asc', 'Soil_Moisture_desc'])
    merged_ndvi = merged_ndvi.dropna(subset=['asc_25km_16d_NDVI', 'desc_25km_16d_NDVI'])
    merged_lst = merged_lst.dropna(subset=['asc_LST_Day_25km_Celsius', 'desc_LST_Night_25km_Celsius'])

    ###########################################################################
    # Merge the DataFrames on 'time', 'lat', and 'lon' columns
#     print("Merged_df1 columns:", merged_df1.columns)
#     print("Merged_df2 columns:", merged_df2.columns)
    merged_df = pd.merge(merged_df1, merged_df2, on=['time', 'lat', 'lon'], how='outer')
#     print("Merged DataFrame shape:", merged_df.shape)
#     print(merged_df.head())

    # Prepare features and target
    x = merged_df1[['asc_LST_Day_25km_Celsius', 'asc_25km_16d_NDVI']]
    y = merged_df2[['Soil_Moisture_asc']]
    x.to_csv('x.csv')
    y.to_csv('y.csv')

    # Trim the two datasets to have the same dimensions
    min_length = min(x.shape[0], y.shape[0])
    x_trimmed = x.iloc[:min_length, :]
    y_trimmed = y[:min_length]

    # Split the data into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x_trimmed, y_trimmed, test_size=0.2, random_state=42)
    x_train.to_csv('x_train.csv')
    y_train.to_csv('y_train.csv')

    # Drop rows with NaN values in either X_train or y_train
    x_train = x_train.dropna().reset_index(drop=True)
    y_train = y_train.dropna().reset_index(drop=True)
    
    # Reset the index for both X_train and y_train
    x_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    
    # Align X_train and y_train by index
    x_train = x_train.loc[y_train.index]

    # Check if there are still NaN values in X_train or y_train
    if x_train.isnull().values.any() or y_train.isnull().values.any():
        print("There are still NaN values after dropping rows. Please handle them accordingly.")
        return None, None, None
    else:
        # Create a polynomial regression model with degree 2
        n = 2
        poly_reg_model = make_pipeline(PolynomialFeatures(degree=n), LinearRegression())

        # Train the model
        poly_reg_model.fit(x_train, y_train)

        # Evaluate the model
        train_score = poly_reg_model.score(x_train, y_train)
        test_score = poly_reg_model.score(x_test, y_test)

        # Make predictions
        predicted_sm = poly_reg_model.predict(x)
        print("Length of predicted_sm:", len(predicted_sm))

        # Extracting the PolynomialFeatures transformer from the pipeline
        poly_features = poly_reg_model.named_steps['polynomialfeatures']
        poly_feature_names = poly_features.get_feature_names_out(input_features=['asc_LST_Day_25km_Celsius', 'asc_25km_16d_NDVI'])
        print("Polynomial feature names:", poly_feature_names)

        # Assign predicted soil moisture values to the DataFrame
        merged_df = merged_df.dropna(subset=['asc_LST_Day_25km_Celsius']).reset_index(drop=True)
        merged_df = merged_df.dropna(subset=['asc_25km_16d_NDVI']).reset_index(drop=True)
        merged_df.reset_index(drop=True, inplace=True)

        # Trim the predicted_sm array to match the length of the DataFrame index
        trimmed_predicted_sm = predicted_sm[:len(merged_df)]
        merged_df['predicted_sm'] = trimmed_predicted_sm
        merged_df.to_csv('predicted_sm.csv')

        # Print model score
        print("Training Score:", train_score)
        print("Testing Score:", test_score)

        # Prepare feature names
        feature_names = ['Intercept', 'asc_LST_Day_25km_Celsius', 'asc_25km_16d_NDVI']
        coefficients = poly_reg_model.named_steps['linearregression'].coef_
        intercept = poly_reg_model.named_steps['linearregression'].intercept_[0]

        # Create the equation string
        equation = "y = "
        intercept_coef = coefficients[0][0]
        equation += f"({intercept_coef:.4f}) + "
        for i, coef in enumerate(coefficients[0][1:], start=1):
            if i < len(feature_names):
                equation += f"({coef:.4f} * {feature_names[i]}) + "
            else:
                equation += f"({coef:.4f} * X{i}) + "
        equation = equation[:-3]
        print("Equation:", equation)
        print("Coefficients:", coefficients)
        print("Intercept:", intercept)

        # Handle NaN values in train_score and test_score
        if not np.isnan(train_score) and not np.isnan(test_score):
            train_score = train_score
            test_score = test_score

        # Convert merged_df predicted_sm DataFrame to xarray for map plotting
        merged_df_reset = merged_df.reset_index()
        merged_df_reset = merged_df_reset.drop_duplicates(subset=['lat', 'lon'])
        merged_df_xr = xr.Dataset.from_dataframe(merged_df_reset.set_index(['lat', 'lon']))

        # Create a map plot using hvplot
        plot = merged_df_xr.hvplot.scatter(x='lon', y='lat', c='predicted_sm', title='Predicted Soil Moisture')
        hvplot.show(plot)

        return equation, train_score, test_score

## Function Initiation
## Input data Ghana over Ghana
smos_asc_file = xr.open_dataset('F:\RWESCK\PHD RESEARCH\Datasets\SMOS\smos_l3_data\smos_l3_files\merged\gh_subset/smos_asc_sub.nc')   ##smos_data.nc')
smos_desc_file = xr.open_dataset('F:\RWESCK\PHD RESEARCH\Datasets\SMOS\smos_l3_data\smos_l3_files\merged\gh_subset/smos_desc_sub.nc')   ##smos_data.nc')
ndvi_asc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/RS_MOD13A2.061_25km_aid0001.nc')
ndvi_desc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/RS_MYD13A2.061_25km_aid0001.nc')
lst_asc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/RS_MOD11A1.061_25km_aid0001.nc')
lst_desc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/RS_MYD11A1.061_25km_aid0001.nc')

# ## Input data over ROI (North)
# smos_asc_file = xr.open_dataset('F:\RWESCK\PHD RESEARCH\Datasets\SMOS\smos_l3_data\smos_l3_files\merged\gh_subset/asc_roi_smos.nc')   ##smos_data.nc')
# smos_desc_file = xr.open_dataset('F:\RWESCK\PHD RESEARCH\Datasets\SMOS\smos_l3_data\smos_l3_files\merged\gh_subset/desc_roi_smos.nc')   ##smos_data.nc')
# ndvi_asc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/roi_RS_MOD13A2.061_25km_aid0001.nc')
# ndvi_desc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/roi_RS_MYD13A2.061_25km_aid0001.nc')
# lst_asc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/roi_RS_MOD11A1.061_25km_aid0001.nc')
# lst_desc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/roi_RS_MYD11A1.061_25km_aid0001.nc')

# Load shapefiles
shapefile_path = 'F:/RWESCK/PHD RESEARCH/Datasets/GHA_adm/GHA_adm01.shp' 
ghana_shp_path = 'F:/RWESCK/PHD RESEARCH/Datasets/GHA_adm/GHA_adm01.shp'  
north_sect_path = 'F:/RWESCK/PHD RESEARCH/Datasets/Ghana-Shapefile-New/Gh_north.shp'

##V2 Initiate Model
poly_reg_model_v2, train_score, test_score = develop_regression_model(smos_asc_file, smos_desc_file, ndvi_asc_file, lst_asc_file)

if poly_reg_model_v2 is not None:
    print(f"Model trained successfully with train score: {train_score} and test score: {test_score}")
else:
    print("Model training failed.")

In [ ]:
### Save the trained regression model to a file
joblib.dump(poly_reg_model_v2, 'poly_reg_model.pkl')

In [ ]:
############################################################

In [ ]:
#### Apply model to merged data

# Load the saved regression model from the file
loaded_model = joblib.load('poly_reg_model.pkl')

In [ ]:
# Load the necessary libraries
import xarray as xr
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

In [ ]:
# Load the trained regression model
poly_reg_model = joblib.load('poly_reg_model.pkl')

# Load the NDVI and LST files
# ndvi_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/RS_MOD13A2.061_25km_aid0001.nc')
# lst_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/RS_MOD11A1.061_25km_aid0001.nc')
## Input data
smos_asc_file = xr.open_dataset('F:\RWESCK\PHD RESEARCH\Datasets\SMOS\smos_l3_data\smos_l3_files\merged\gh_subset/smos_asc_sub.nc')   ##smos_data.nc')
smos_desc_file = xr.open_dataset('F:\RWESCK\PHD RESEARCH\Datasets\SMOS\smos_l3_data\smos_l3_files\merged\gh_subset/smos_desc_sub.nc')   ##smos_data.nc')
ndvi_asc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/RS_MOD13A2.061_25km_aid0001.nc')
ndvi_desc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/RS_MYD13A2.061_25km_aid0001.nc')
lst_asc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/RS_MOD11A1.061_25km_aid0001.nc')
lst_desc_file = xr.open_dataset('F:/RWESCK/PHD RESEARCH/Datasets/Downscaling/MODIS/new_dl/RS_MYD11A1.061_25km_aid0001.nc')



# Extract the features (NDVI and LST)
ndvi_data = ndvi_asc_file['asc_25km_16d_NDVI']
lst_data = lst_asc_file['asc_LST_Day_25km_Celsius']

In [ ]:
# Realigning datasets (LST is daily and NDVi is 16 day average)
# Reindex NDVI data using the time points of LST data
ndvi_aligned = ndvi_data.reindex(time=lst_data['time'])

# Forward fill NDVI values to fill missing values for each day of LST
ndvi_aligned = ndvi_aligned.ffill(dim='time')

# Select variables to use
modis_ndvi = ndvi_aligned#['_1_km_16_days_NDVI']
modis_lst = lst_data#['LST_Day_1km_Celsius']

# Convert to DataFrame
ndvi_df = modis_ndvi.to_dataframe().reset_index()
lst_df = modis_lst.to_dataframe().reset_index()
    
# Merge DataFrames on 'time', 'lat', and 'lon'
merged_df = pd.merge(lst_df, ndvi_df, on=['time', 'lat', 'lon'])

 # Drop rows with NaN values in a specific column
merged_df = merged_df.dropna(subset=['asc_LST_Day_25km_Celsius'])
print(merged_df)

In [ ]:
#### Estimate SM using the trained model
# Prepare features for prediction
x_all = merged_df[['asc_LST_Day_25km_Celsius', 'asc_25km_16d_NDVI']]

# Make predictions
predicted_sm_all = poly_reg_model.predict(x_all)

# Add the predicted soil moisture values to the DataFrame
merged_df['downscaled_sm'] = predicted_sm_all

# Save the predicted soil moisture to a suitable format (e.g., CSV)
merged_df.to_csv('downscaled_soil_moisture_all.csv')
print(merged_df)

In [ ]:
# Load Ghana shapefile
ghana_shp = gpd.read_file('F:/RWESCK/PHD RESEARCH/Datasets/GHA_adm/GHA_adm01.shp')  # Replace with the actual path

# Plot the predicted soil moisture map for all data points
plt.figure(figsize=(10, 6))
plt.scatter(merged_df['lon'], merged_df['lat'], c=merged_df['downscaled_sm'], cmap='viridis')
plt.colorbar(label='Downscaled SM')

# Overlay Ghana shapefile
ghana_shp.plot(ax=plt.gca(), facecolor='none', edgecolor='red', linewidth=1.0)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Downscaled SM for All Data')
plt.grid(True)
plt.show()

In [ ]:
print(merged_df)
print(merged_df.shape)

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np

# Extract necessary data
lons = merged_df['lon'].values
lats = merged_df['lat'].values
soil_moisture = merged_df['downscaled_sm'].values

# Create mesh grid for plotting
lon_mesh, lat_mesh = np.meshgrid(lons, lats)

# Create a figure and axis with PlateCarree projection
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the downscaled soil moisture data
mesh = ax.pcolormesh(
    lon_mesh,  # Mesh grid for longitude values
    lat_mesh,  # Mesh grid for latitude values
    soil_moisture,  # Soil moisture values
    cmap='viridis',  # Color map
    transform=ccrs.PlateCarree(),  # Coordinate reference system
)

# Add coastlines and gridlines
ax.coastlines()
ax.gridlines()

# Add colorbar
cbar = plt.colorbar(mesh, ax=ax, orientation='vertical', shrink=0.8)
cbar.set_label('Downscaled Soil Moisture')

# Add title
plt.title('Map Plot of Downscaled Soil Moisture')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np

# Assuming you have latitude and longitude arrays
lats = merged_df['lat'].unique()
lons = merged_df['lon'].unique()

# Create mesh grid for plotting
lon_mesh, lat_mesh = np.meshgrid(lons, lats)

# Extract necessary data
soil_moisture = merged_df['downscaled_sm'].values
soil_moisture_2d = soil_moisture.reshape(lon_mesh.shape)


# Create mesh grid for plotting using downsampled values
lon_mesh, lat_mesh = np.meshgrid(lons_downsampled, lats_downsampled)

# Create a figure and axis with PlateCarree projection
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the downscaled soil moisture data
mesh = ax.pcolormesh(
    lon_mesh,  # Mesh grid for longitude values
    lat_mesh,  # Mesh grid for latitude values
    soil_moisture_2d,  # 2D array of soil moisture values
    cmap='viridis',  # Color map
    transform=ccrs.PlateCarree(),  # Coordinate reference system
)


# Add coastlines and gridlines
ax.coastlines()
ax.gridlines()

# Add colorbar
cbar = plt.colorbar(mesh, ax=ax, orientation='vertical', shrink=0.8)
cbar.set_label('Downscaled Soil Moisture')

# Add title
plt.title('Map Plot of Downscaled Soil Moisture')

# Show the plot
plt.show()


In [ ]:
# 1km_sm = 
print(merged_df_xr['downscaled_sm'])
print(merged_df_xr['lon', 'lat'])

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

# Define a function to plot data for selected date
def plot_data(selected_date):
    # Filter data for the selected date
    selected_data = merged_df_xr.sel(time=selected_date)
    
    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.scatter(selected_data['lon'], selected_data['lat'], c=selected_data['downscaled_sm'], cmap='viridis')
    plt.colorbar(label='Downscaled Soil Moisture')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(f'Downscaled Soil Moisture for {selected_date}')
    plt.grid(True)
    plt.show()

# Get unique dates
unique_dates = merged_df_xr['time'].values.tolist()

# Create dropdown widget
date_dropdown = widgets.Dropdown(
    options=unique_dates,
    description='Select Date:',
    disabled=False,
)

# Define widget event handler
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_date = change['new']
        plot_data(selected_date)

# Attach event handler to dropdown widget
date_dropdown.observe(on_change)

# Display the dropdown widget
display(date_dropdown)


In [ ]:
#     # Create a map plot of 25km predicted SM estimates
#Create a figure and axis
plt.figure(figsize=(10, 6))
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})

# Plot the variable on the map
merged_df_xr['downscaled_sm'].plot(ax=ax, cmap='viridis', transform=ccrs.PlateCarree())
    
# Overlay the shapefile of Ghana
ghana_shp.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=1, transform=ccrs.PlateCarree())

# Add coastlines and gridlines
ax.gridlines()
    
# Add a title
plt.title('1km Downscaled SM Map')
    # Add a colorbar
#plt.colorbar(label='Soil Moisture')
#plt.colorbar(scatter, label='Predicted Soil Moisture')


######################################################################

1. The above predicted SM is at 25km resolution
2. Now use the model derived to predict SM at the MODIS 1km resolution 
3. Now u have your L4 Gh SM estimates at 1km resolution